# Import Libraries

In [1]:
# Data
import pandas as pd
import numpy as np
import time

# Ontology
!pip install owlready2 
#import owlready2 as owl
from owlready2 import *
import re  # To separate words based on capital letters in onto classes & to split search queries
!pip install EMMOntoPy #Special EMMO package
# from ontopy import get_ontology

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


## Ontology

In [14]:
"""Import Ontology & Select classes"""

# write the location to the ontology to the onto_path list
onto_path = ["https://raw.githubusercontent.com/hendelhendel/FAIR_Battery/main/Ontology/test.owl"]
    # Note that all the ontology files has to be owl files.
    
# Import ontology
try:
    onto = get_ontology(onto_path[0]).load()
except:
    pass 

onto = get_ontology(onto_path[0]).load()

# Collecting classes from ontology in a list
class_raw = list(get_ontology(onto_path[0]).load().classes())

# select classes by Prefix, suffix, nametags
tag = 'ElectrochemicalFlowCell'
prefix = 'electrochemistry.'

ClassCleaner = lambda x : re.sub('_',  ' ',\
                                 re.sub(r"(?<=\w)([A-Z])", r" \1", \
                                 str(x).removesuffix(tag).removeprefix(prefix))) \
                                    if (str(x).find(prefix) != -1) else "!EMPTY CLASS" 
                            

class_select = list(map(ClassCleaner, filter(lambda x : (str(x).find(tag) != -1), class_raw)))

print("Your imported ontology with "  + str(len(class_select)) + " classes is ready to use")

Your imported ontology with 33 classes is ready to use


## Process Data

In [28]:
"""Import Zotero Data Base from Github"""

# Import raw data from github repository as dataframe
data_path = 'https://raw.githubusercontent.com/hendelhendel/FAIR_Battery/main/Datamanagement/FlowBattKnowledgebase_1.csv'
df_raw = pd.read_csv(data_path)

In [16]:
"""Data Process Functions"""

# Function Text Cleaner
f_CleanText = lambda text : re.split("\. |\! |\? ", text.lower()) if (type(text) == str)  else ["Not Available"]
    # Cleans text split scentices.
    # Output is of strings per for every text inputed. 
    # Example: Input = sentence1.sentence2!sentence3? --> Output = [sentence1,sentence2,sentence3]
    
    
# Function Keyword Search    
f_SearchKeyword = lambda text, keyword : text if (text.find(keyword) != -1) else None 
    # Searches text for keyword
    # Input: text = string or list of strings, keyword = string or list strings
    # Example: 
        # text = [sentence1,sentence2,sentence_A,sentence_B]
        # keyword = len(text)*[A]
        # Output  = [0,0,sentence_A,0]
        
# Function Scan lists of lists with text
f_ScanList = lambda text, keyword: list(map(f_SearchKeyword, text , len(text)*[keyword]))  
    # Example: 
        # text = [[sentence1,sentence2],[sentence_A,sentence_B]]
        # keyword = [A]
        # Output  = [[0,0],[sentence_A,0]]
                                        
#f_ScanList = lambda text, keyword: f_SearchKeyword(text, 'keyword') #if (type(text) == str) else "appel"
#f_ScanList = lambda text, keyword: len(text) if (type(text) == str) else "appel"

In [34]:
"""Process Data"""
start = time.time() # Measure time

# New data frame to store processed data
df_processed = df_raw
SourceText = "Abstract Note" #Column name text source
ProcessedText = "Clean Abstract Note" # Column name processed text

# clean abstract data 
df_processed[ProcessedText] = df_raw[SourceText].map(f_CleanText)

# function to search data for ontology classes !! depends on df_processed !!
f_OntoSearch = lambda onto : list(map(f_ScanList, df_processed[ProcessedText], len(df_processed[ProcessedText])*[onto.lower()]))

# Search data ontology class and store in df
def f_AddSearchResults(df, ontology):
    for onto in ontology: 
        df[str(onto)] = f_OntoSearch(onto)

f_AddSearchResults(df_processed, class_select)

end = time.time() # Measure time
calc_time = end-start  # Measure time

print("Your " + str(len(df_processed)) + " articles are searched on "  + str(len(class_select)) + " ontology classes in " + str(calc_time) + " seconds.")
print("type df_processed to view data frame")

# Store processed data
df_processed.to_csv('ProcessedData.csv')


Your 5644 articles are searched on 33 ontology classes in 0.32325148582458496 seconds.
type df_processed to view data frame


In [23]:
df_processed["Pump"][30]

['an analysis is presented of the losses occurring in a kw-class vanadium redox flow battery due to species crossover, shunt currents, hydraulic pressure drops and pumping, in addition to cell overpotentials',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 'moreover, it was found that pumps and their motors can affect significantly the efficiency of a kw-class vanadium redox flow battery system.']

In [22]:
len(df_processed)

5644